In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import re

import gffutils
import numpy as np
import pandas as pd
import pybedtools
import six

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
outrigger_folder = '{}/outrigger_index'.format(folder)
csv_folder = '{}/csvs_for_paper/'.format(folder)

# se_df = pd.read_csv('{}/exons_to_junctions_se.csv'.format(folder), index_col=0)
se_df = pd.read_csv('{}/events/se.csv'.format(outrigger_folder), index_col=0)

# renamer = dict((x, x.replace('junction_exon', 'junction_exons')) for x in se_df)
# print renamer
# se_df = se_df.rename(columns=renamer)
print se_df.shape
se_df.head()

(40534, 7)


/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


,junction13,junction12,junction23,criteria_full,event_id,criteria,criteria_additional
0,chr10:100189400-100190327:-,chr10:100189647-100190327:-,chr10:100189400-100189547:-,"best,appris_principal",exon:chr10:100190328-100190427:-@exon:chr10:10...,best,appris_principal
1,chr10:100190428-100193696:-,chr10:100191049-100193696:-,chr10:100190428-100190887:-,only one,exon:chr10:100193697-100193848:-@exon:chr10:10...,only one,NaN
2,chr10:100193849-100195391:-,chr10:100195172-100195391:-,chr10:100193849-100195028:-,"best,appris_principal",exon:chr10:100195392-100195529:-@exon:chr10:10...,best,appris_principal
3,chr10:101163392-101165512:-,chr10:101163632-101165512:-,chr10:101163392-101163480:-,only one,exon:chr10:101165513-101165617:-@exon:chr10:10...,only one,NaN
4,chr10:101419346-101421202:+,chr10:101419346-101419618:+,chr10:101419722-101421202:+,only one,exon:chr10:101419263-101419345:+@exon:chr10:10...,only one,NaN


Double check that there are no splicing events which share junctions but not flanking exons. I.e. if you group by the junctions, you should only get one event

In [2]:
grouped = se_df.groupby(['junction13', 'junction12', 'junction23'])
size = grouped.size()
(size != 1).sum()

0

In [3]:
splicing = pd.read_csv('{}/splicing.csv'.format(csv_folder), index_col=0)
splicing.shape

(242, 26379)

In [4]:
se_exons = se_df.event_id.str.split('@').apply(pd.Series).rename(columns=dict((i, 'exon{}'.format(i+1)) for i in range(3)))
se_df = pd.concat([se_df, se_exons], axis=1)
six.print_(se_df.shape)
se_df = se_df.loc[se_df.event_id.isin(splicing.columns)]
six.print_(se_df.shape)
se_df.head()

(40534, 10)
(23314, 10)


,junction13,junction12,junction23,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3
0,chr10:100189400-100190327:-,chr10:100189647-100190327:-,chr10:100189400-100189547:-,"best,appris_principal",exon:chr10:100190328-100190427:-@exon:chr10:10...,best,appris_principal,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-
1,chr10:100190428-100193696:-,chr10:100191049-100193696:-,chr10:100190428-100190887:-,only one,exon:chr10:100193697-100193848:-@exon:chr10:10...,only one,NaN,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-
2,chr10:100193849-100195391:-,chr10:100195172-100195391:-,chr10:100193849-100195028:-,"best,appris_principal",exon:chr10:100195392-100195529:-@exon:chr10:10...,best,appris_principal,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-
3,chr10:101163392-101165512:-,chr10:101163632-101165512:-,chr10:101163392-101163480:-,only one,exon:chr10:101165513-101165617:-@exon:chr10:10...,only one,NaN,exon:chr10:101165513-101165617:-,exon:chr10:101163481-101163631:-,exon:chr10:101163226-101163391:-
4,chr10:101419346-101421202:+,chr10:101419346-101419618:+,chr10:101419722-101421202:+,only one,exon:chr10:101419263-101419345:+@exon:chr10:10...,only one,NaN,exon:chr10:101419263-101419345:+,exon:chr10:101419619-101419721:+,exon:chr10:101421203-101421385:+


## MXE

In [5]:
mxe_df = pd.read_csv('{}/events/mxe.csv'.format(outrigger_folder))
print mxe_df.shape
mxe_df.head()

(13217, 9)


,Unnamed: 0,junction13,junction34,junction12,junction24,criteria_full,event_id,criteria,criteria_additional
0,0,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,one event with gencode transcripts,exon:chr10:102003455-102003534:-@exon:chr10:10...,one event with gencode transcripts,NaN
1,1,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,"random,no gencode transcripts",exon:chr10:102013178-102013296:-@exon:chr10:10...,random,no gencode transcripts
2,2,chr10:102016234-102020722:-,chr10:102013297-102016018:-,chr10:102020077-102020722:-,chr10:102013297-102019974:-,"random,no good tags",exon:chr10:102020723-102020801:-@exon:chr10:10...,random,no good tags
3,3,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020077-102021735:-,chr10:102013297-102019974:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN
4,4,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020786-102021735:-,chr10:102013297-102020722:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN


In [6]:
mxe_exons = mxe_df.event_id.str.split('@').apply(pd.Series).rename(columns=dict((i, 'exon{}'.format(i+1)) for i in range(4)))
mxe_df = pd.concat([mxe_df, mxe_exons], axis=1)
mxe_df.head()

,Unnamed: 0,junction13,junction34,junction12,junction24,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3,exon4
0,0,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,one event with gencode transcripts,exon:chr10:102003455-102003534:-@exon:chr10:10...,one event with gencode transcripts,NaN,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,exon:chr10:101996607-101996726:-,exon:chr10:101995424-101995521:-
1,1,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,"random,no gencode transcripts",exon:chr10:102013178-102013296:-@exon:chr10:10...,random,no gencode transcripts,exon:chr10:102013178-102013296:-,exon:chr10:102010005-102010089:-,exon:chr10:102003455-102003534:-,exon:chr10:101997940-101997988:-
2,2,chr10:102016234-102020722:-,chr10:102013297-102016018:-,chr10:102020077-102020722:-,chr10:102013297-102019974:-,"random,no good tags",exon:chr10:102020723-102020801:-@exon:chr10:10...,random,no good tags,exon:chr10:102020723-102020801:-,exon:chr10:102019975-102020076:-,exon:chr10:102016019-102016233:-,exon:chr10:102013178-102013296:-
3,3,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020077-102021735:-,chr10:102013297-102019974:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN,exon:chr10:102021736-102021820:-,exon:chr10:102019975-102020076:-,exon:chr10:102016019-102016233:-,exon:chr10:102013178-102013296:-
4,4,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020786-102021735:-,chr10:102013297-102020722:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN,exon:chr10:102021736-102021820:-,exon:chr10:102020723-102020785:-,exon:chr10:102016019-102016233:-,exon:chr10:102013178-102013296:-


In [7]:
from outrigger.region import Region

def get_gene_attributes(df, exon_cols, db):
    df['gencode_id'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[db[i].attributes['gene_id'] for i in x]))), axis=1)
    df['transcript_id'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[db[i].attributes['transcript_id'] for i in x]))), axis=1)
    df['gene_name'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[db[i].attributes['gene_name'] for i in x]))), axis=1)
    df['ensembl_id'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[map(lambda y: y.split('.')[0], 
                                                     db[i].attributes['gene_id']) for i in x]))), 
        axis=1)
    
    for exon_col in exon_cols:
        df['{}_region'.format(exon_col)] = df[exon_col].map(Region)
        df['{}_length'.format(exon_col)] = df['{}_region'.format(exon_col)].map(len)
    df['strand'] = df[exon_cols[0]].str[-1]
    return df

## Annotate events with gene ids and exon lengths

In [8]:
mxe_exon_cols = ['exon{}'.format(i) for i in range(1, 5)]
print(mxe_exon_cols)
mxe_df_annotated = get_gene_attributes(mxe_df, mxe_exon_cols, v19db)

['exon1', 'exon2', 'exon3', 'exon4']


In [9]:
se_exon_cols = ['exon{}'.format(i) for i in range(1, 4)]
print(se_exon_cols)
se_df_annotated = get_gene_attributes(se_df, se_exon_cols, v19db)

['exon1', 'exon2', 'exon3']


### Spot-check some genes

In [10]:
mxe_df_annotated.loc[mxe_df_annotated.gene_name == 'PKM']

,Unnamed: 0,junction13,junction34,junction12,junction24,criteria_full,event_id,criteria,criteria_additional,exon1,...,ensembl_id,exon1_region,exon1_length,exon2_region,exon2_length,exon3_region,exon3_length,exon4_region,exon4_length,strand
2294,2294,chr15:72494962-72499068:-,chr15:72492997-72494794:-,chr15:72495530-72499068:-,chr15:72492997-72495362:-,one event with gencode transcripts,exon:chr15:72499069-72499221:-@exon:chr15:7249...,one event with gencode transcripts,NaN,exon:chr15:72499069-72499221:-,...,ENSG00000067225,exon:chr15:72499069-72499221:-,153,exon:chr15:72495363-72495529:-,167,exon:chr15:72494795-72494961:-,167,exon:chr15:72492815-72492996:-,182,-


In [11]:
se_df_annotated.loc[se_df_annotated.gene_name == 'RPS24']

,junction13,junction12,junction23,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3,...,transcript_id,gene_name,ensembl_id,exon1_region,exon1_length,exon2_region,exon2_length,exon3_region,exon3_length,strand
1351,chr10:79797063-79799961:+,chr10:79797063-79797722:+,chr10:79797741-79799961:+,only one,exon:chr10:79796952-79797062:+@exon:chr10:7979...,only one,NaN,exon:chr10:79796952-79797062:+,exon:chr10:79797723-79797740:+,exon:chr10:79799962-79799983:+,...,"ENST00000485708.2,ENST00000476545.1,ENST000003...",RPS24,ENSG00000138326,exon:chr10:79796952-79797062:+,111,exon:chr10:79797723-79797740:+,18,exon:chr10:79799962-79799983:+,22,+
1352,chr10:79797063-79800372:+,chr10:79797063-79797722:+,chr10:79797741-79800372:+,"best,appris_principal",exon:chr10:79796952-79797062:+@exon:chr10:7979...,best,appris_principal,exon:chr10:79796952-79797062:+,exon:chr10:79797723-79797740:+,exon:chr10:79800373-79800455:+,...,"ENST00000485708.2,ENST00000476545.1,ENST000004...",RPS24,ENSG00000138326,exon:chr10:79796952-79797062:+,111,exon:chr10:79797723-79797740:+,18,exon:chr10:79800373-79800455:+,83,+
1353,chr10:79797063-79800372:+,chr10:79797063-79799961:+,chr10:79799984-79800372:+,"best,appris_principal",exon:chr10:79796952-79797062:+@exon:chr10:7979...,best,appris_principal,exon:chr10:79796952-79797062:+,exon:chr10:79799962-79799983:+,exon:chr10:79800373-79800471:+,...,"ENST00000485708.2,ENST00000476545.1,ENST000003...",RPS24,ENSG00000138326,exon:chr10:79796952-79797062:+,111,exon:chr10:79799962-79799983:+,22,exon:chr10:79800373-79800471:+,99,+
1354,chr10:79797741-79800372:+,chr10:79797741-79799961:+,chr10:79799984-79800372:+,one event with gencode transcripts,exon:chr10:79797723-79797740:+@exon:chr10:7979...,one event with gencode transcripts,NaN,exon:chr10:79797723-79797740:+,exon:chr10:79799962-79799983:+,exon:chr10:79800373-79800428:+,...,"ENST00000485708.2,ENST00000360830.4",RPS24,ENSG00000138326,exon:chr10:79797723-79797740:+,18,exon:chr10:79799962-79799983:+,22,exon:chr10:79800373-79800428:+,56,+


In [12]:
se_df_annotated.loc[se_df_annotated.gene_name == 'EIF4A2']

,junction13,junction12,junction23,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3,...,transcript_id,gene_name,ensembl_id,exon1_region,exon1_length,exon2_region,exon2_length,exon3_region,exon3_length,strand
28066,chr3:186502486-186503671:+,chr3:186502486-186502750:+,chr3:186502891-186503671:+,"best,appris_principal",exon:chr3:186502353-186502485:+@exon:chr3:1865...,best,appris_principal,exon:chr3:186502353-186502485:+,exon:chr3:186502751-186502890:+,exon:chr3:186503672-186503840:+,...,"ENST00000323963.5,ENST00000356531.5,ENST000004...",EIF4A2,ENSG00000156976,exon:chr3:186502353-186502485:+,133,exon:chr3:186502751-186502890:+,140,exon:chr3:186503672-186503840:+,169,+
28067,chr3:186502486-186503952:+,chr3:186502486-186503671:+,chr3:186503841-186503952:+,one event with gencode transcripts,exon:chr3:186502353-186502485:+@exon:chr3:1865...,one event with gencode transcripts,NaN,exon:chr3:186502353-186502485:+,exon:chr3:186503672-186503840:+,exon:chr3:186503953-186504062:+,...,"ENST00000323963.5,ENST00000356531.5,ENST000004...",EIF4A2,ENSG00000156976,exon:chr3:186502353-186502485:+,133,exon:chr3:186503672-186503840:+,169,exon:chr3:186503953-186504062:+,110,+
28068,chr3:186502486-186504290:+,chr3:186502486-186503952:+,chr3:186504063-186504290:+,only one,exon:chr3:186502353-186502485:+@exon:chr3:1865...,only one,NaN,exon:chr3:186502353-186502485:+,exon:chr3:186503953-186504062:+,exon:chr3:186504291-186504434:+,...,"ENST00000323963.5,ENST00000356531.5,ENST000004...",EIF4A2,ENSG00000156976,exon:chr3:186502353-186502485:+,133,exon:chr3:186503953-186504062:+,110,exon:chr3:186504291-186504434:+,144,+
28069,chr3:186504063-186504915:+,chr3:186504063-186504290:+,chr3:186504435-186504915:+,"best,appris_principal",exon:chr3:186503953-186504062:+@exon:chr3:1865...,best,appris_principal,exon:chr3:186503953-186504062:+,exon:chr3:186504291-186504434:+,exon:chr3:186504916-186505053:+,...,"ENST00000492144.1,ENST00000323963.5,ENST000003...",EIF4A2,ENSG00000156976,exon:chr3:186503953-186504062:+,110,exon:chr3:186504291-186504434:+,144,exon:chr3:186504916-186505053:+,138,+
28070,chr3:186505054-186505591:+,chr3:186505054-186505267:+,chr3:186505374-186505591:+,"best,appris_principal",exon:chr3:186504916-186505053:+@exon:chr3:1865...,best,appris_principal,exon:chr3:186504916-186505053:+,exon:chr3:186505268-186505373:+,exon:chr3:186505592-186505671:+,...,"ENST00000492144.1,ENST00000356531.5,ENST000003...",EIF4A2,ENSG00000156976,exon:chr3:186504916-186505053:+,138,exon:chr3:186505268-186505373:+,106,exon:chr3:186505592-186505671:+,80,+
28071,chr3:186505054-186505591:+,chr3:186505054-186505283:+,chr3:186505374-186505591:+,"best,appris_principal",exon:chr3:186504916-186505053:+@exon:chr3:1865...,best,appris_principal,exon:chr3:186504916-186505053:+,exon:chr3:186505284-186505373:+,exon:chr3:186505592-186505671:+,...,"ENST00000492144.1,ENST00000356531.5,ENST000003...",EIF4A2,ENSG00000156976,exon:chr3:186504916-186505053:+,138,exon:chr3:186505284-186505373:+,90,exon:chr3:186505592-186505671:+,80,+
28072,chr3:186505374-186506098:+,chr3:186505374-186505591:+,chr3:186505672-186506098:+,"random,no good tags",exon:chr3:186505268-186505373:+@exon:chr3:1865...,random,no good tags,exon:chr3:186505268-186505373:+,exon:chr3:186505592-186505671:+,exon:chr3:186506099-186506209:+,...,"ENST00000323963.5,ENST00000356531.5,ENST000004...",EIF4A2,ENSG00000156976,exon:chr3:186505268-186505373:+,106,exon:chr3:186505592-186505671:+,80,exon:chr3:186506099-186506209:+,111,+
28073,chr3:186505672-186506913:+,chr3:186505672-186506098:+,chr3:186506206-186506913:+,"best,appris_principal",exon:chr3:186505592-186505671:+@exon:chr3:1865...,best,appris_principal,exon:chr3:186505592-186505671:+,exon:chr3:186506099-186506205:+,exon:chr3:186506914-186507689:+,...,"ENST00000323963.5,ENST00000356531.5,ENST000004...",EIF4A2,ENSG00000156976,exon:chr3:186505592-186505671:+,80,exon:chr3:186506099-186506205:+,107,exon:chr3:186506914-186507689:+,776,+
28074,chr3:186505672-186506913:+,

## Get intron lengths

In [13]:
positive_strand = se_df_annotated.strand == '+'
se_df_annotated['intron_length'] = np.nan


se_df_annotated.loc[positive_strand, 'intron_length'] = \
    se_df_annotated.loc[positive_strand].apply(lambda x: x.exon3_region._start - x.exon1_region._stop - 1, axis=1)
se_df_annotated.loc[~positive_strand, 'intron_length'] = \
    se_df_annotated.loc[~positive_strand].apply(lambda x: x.exon1_region._start - x.exon3_region._stop - 1, axis=1)
se_df_annotated.head()

,junction13,junction12,junction23,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3,...,gene_name,ensembl_id,exon1_region,exon1_length,exon2_region,exon2_length,exon3_region,exon3_length,strand,intron_length
0,chr10:100189400-100190327:-,chr10:100189647-100190327:-,chr10:100189400-100189547:-,"best,appris_principal",exon:chr10:100190328-100190427:-@exon:chr10:10...,best,appris_principal,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,...,HPS1,ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,-,928
1,chr10:100190428-100193696:-,chr10:100191049-100193696:-,chr10:100190428-100190887:-,only one,exon:chr10:100193697-100193848:-@exon:chr10:10...,only one,NaN,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,...,HPS1,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,-,3269
2,chr10:100193849-100195391:-,chr10:100195172-100195391:-,chr10:100193849-100195028:-,"best,appris_principal",exon:chr10:100195392-100195529:-@exon:chr10:10...,best,appris_principal,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,...,HPS1,ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,-,1543
3,chr10:101163392-101165512:-,chr10:101163632-101165512:-,chr10:101163392-101163480:-,only one,exon:chr10:101165513-101165617:-@exon:chr10:10...,only one,NaN,exon:chr10:101165513-101165617:-,exon:chr10:101163481-101163631:-,exon:chr10:101163226-101163391:-,...,GOT1,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,-,2121
4,chr10:101419346-101421202:+,chr10:101419346-101419618:+,chr10:101419722-101421202:+,only one,exon:chr10:101419263-101419345:+@exon:chr10:10...,only one,NaN,exon:chr10:101419263-101419345:+,exon:chr10:101419619-101419721:+,exon:chr10:101421203-101421385:+,...,ENTPD7,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,+,1857


In [14]:
positive_strand = mxe_df_annotated.strand == '+'
mxe_df_annotated['intron_length'] = np.nan


mxe_df_annotated.loc[positive_strand, 'intron_length'] = \
    mxe_df_annotated.loc[positive_strand].apply(lambda x: x.exon4_region._start - x.exon1_region._stop - 1, axis=1)
mxe_df_annotated.loc[~positive_strand, 'intron_length'] = \
    mxe_df_annotated.loc[~positive_strand].apply(lambda x: x.exon1_region._start - x.exon4_region._stop - 1, axis=1)
mxe_df_annotated.head()

,Unnamed: 0,junction13,junction34,junction12,junction24,criteria_full,event_id,criteria,criteria_additional,exon1,...,exon1_region,exon1_length,exon2_region,exon2_length,exon3_region,exon3_length,exon4_region,exon4_length,strand,intron_length
0,0,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,one event with gencode transcripts,exon:chr10:102003455-102003534:-@exon:chr10:10...,one event with gencode transcripts,NaN,exon:chr10:102003455-102003534:-,...,exon:chr10:102003455-102003534:-,80,exon:chr10:101997779-101997988:-,210,exon:chr10:101996607-101996726:-,120,exon:chr10:101995424-101995521:-,98,-,7933
1,1,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,"random,no gencode transcripts",exon:chr10:102013178-102013296:-@exon:chr10:10...,random,no gencode transcripts,exon:chr10:102013178-102013296:-,...,exon:chr10:102013178-102013296:-,119,exon:chr10:102010005-102010089:-,85,exon:chr10:102003455-102003534:-,80,exon:chr10:101997940-101997988:-,49,-,15189
2,2,chr10:102016234-102020722:-,chr10:102013297-102016018:-,chr10:102020077-102020722:-,chr10:102013297-102019974:-,"random,no good tags",exon:chr10:102020723-102020801:-@exon:chr10:10...,random,no good tags,exon:chr10:102020723-102020801:-,...,exon:chr10:102020723-102020801:-,79,exon:chr10:102019975-102020076:-,102,exon:chr10:102016019-102016233:-,215,exon:chr10:102013178-102013296:-,119,-,7426
3,3,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020077-102021735:-,chr10:102013297-102019974:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN,exon:chr10:102021736-102021820:-,...,exon:chr10:102021736-102021820:-,85,exon:chr10:102019975-102020076:-,102,exon:chr10:102016019-102016233:-,215,exon:chr10:102013178-102013296:-,119,-,8439
4,4,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020786-102021735:-,chr10:102013297-102020722:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN,exon:chr10:102021736-102021820:-,...,exon:chr10:102021736-102021820:-,85,exon:chr10:102020723-102020785:-,63,exon:chr10:102016019-102016233:-,215,exon:chr10:102013178-102013296:-,119,-,8439


In [15]:

dfs = []
annotated_dfs = se_df_annotated, mxe_df_annotated
for df in annotated_dfs:
    cols = [x  for x in df.columns if 'region' not in x]
    dfs.append(df[cols])

splicing_feature_data = pd.concat(dfs, ignore_index=True)
splicing_feature_data = splicing_feature_data.set_index('event_id')
print splicing_feature_data.shape
splicing_feature_data.tail()

(36531, 23)


,Unnamed: 0,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,...,gencode_id,gene_name,intron_length,junction12,junction13,junction23,junction24,junction34,strand,transcript_id
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+,13212,only one,NaN,only one,ENSG00000129824,exon:chrY:2710206-2710283:+,78,exon:chrY:2712118-2712298:+,181,exon:chrY:2713687-2713784:+,...,ENSG00000129824.11,RPS4Y1,12357,chrY:2710284-2712117:+,chrY:2710284-2713686:+,NaN,chrY:2712299-2722640:+,chrY:2713785-2722640:+,+,"ENST00000430575.1,ENST00000250784.8"
exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2734834-2734903:+,13213,only one,NaN,only one,ENSG00000129824,exon:chrY:2712118-2712298:+,181,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,...,ENSG00000129824.11,RPS4Y1,22535,chrY:2712299-2713686:+,chrY:2712299-2722640:+,NaN,chrY:2713785-2734833:+,chrY:2722813-2734833:+,+,"ENST00000430575.1,ENST00000250784.8"
exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2734903:+,13214,only one,NaN,only one,ENSG00000129824,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,172,exon:chrY:2733129-2733286:+,...,ENSG00000129824.11,RPS4Y1,21049,chrY:2713785-2722640:+,chrY:2713785-2733128:+,NaN,chrY:2722813-2734833:+,chrY:2733287-2734833:+,+,"ENST00000430575.1,ENST00000477725.1,ENST000002..."
exon:chrY:2803112-2803487:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+,13215,one event with gencode transcripts,NaN,one event with gencode transcripts,ENSG00000067646,exon:chrY:2803112-2803487:+,376,exon:chrY:2821950-2822038:+,89,exon:chrY:2829115-2829687:+,...,ENSG00000067646.7,ZFY,39648,chrY:2803488-2821949:+,chrY:2803488-2829114:+,NaN,chrY:2822039-2843135:+,chrY:2829688-2843135:+,+,"ENST00000443793.1,ENST00000431102.1,ENST000003..."
exon:chrY:2803518-2803810:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+,13216,one event with gencode transcripts,NaN,one event with gencode transcripts,ENSG00000067646,exon:chrY:2803518-2803810:+,293,exon:chrY:2821950-2822038:+,89,exon:chrY:2829115-2829687:+,...,ENSG00000067646.7,ZFY,39325,chrY:2803811-2821949:+,chrY:2803811-2829114:+,NaN,chrY:2822039-2843135:+,chrY:2829688-2843135:+,+,"ENST00000443793.1,ENST00000431102.1,ENST000003..."


In [16]:
splicing_feature_data['exon2_divisible_by_3'] = splicing_feature_data['exon2_length'].astype(int).mod(3) == 0
splicing_feature_data.head()

,Unnamed: 0,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,...,gene_name,intron_length,junction12,junction13,junction23,junction24,junction34,strand,transcript_id,exon2_divisible_by_3
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,NaN,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,...,HPS1,928,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",True
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,NaN,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,...,HPS1,3269,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",False
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,NaN,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,...,HPS1,1543,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,"ENST00000361490.4,ENST00000325103.6,ENST000004...",False
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,NaN,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,...,GOT1,2121,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,"ENST00000543866.1,ENST00000370508.5",False
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,NaN,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,...,ENTPD7,1857,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,ENST00000370489.4,False


In [17]:
splicing_feature_data['exon2_divisible_by_3'].sum()/float(splicing_feature_data.shape[0])

0.35164654676849799

In [18]:
print splicing_feature_data.ensembl_id.str.contains(',').sum()
print splicing_feature_data.shape

2299
(36531, 24)


3,629/53,751 splicing events have multiple gene ids associated with them. Since we want to intersect this information with existing gene annotations, we need a *single* id. To do this coarsely and simply, we will take the first gene ID for intersecting purposes.

In [19]:
pd.options.display.max_columns = 50

In [20]:
splicing_feature_data['one_ensembl_id'] = splicing_feature_data.ensembl_id.str.split(',').str[0]
splicing_feature_data.head()

,Unnamed: 0,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,exon4,exon4_length,gencode_id,gene_name,intron_length,junction12,junction13,junction23,junction24,junction34,strand,transcript_id,exon2_divisible_by_3,one_ensembl_id
event_id,,,,,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,NaN,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,NaN,NaN,ENSG00000107521.14,HPS1,928,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",True,ENSG00000107521
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,NaN,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,NaN,NaN,ENSG00000107521.14,HPS1,3269,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",False,ENSG00000107521
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,NaN,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,NaN,NaN,ENSG00000107521.14,HPS1,1543,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,"ENST00000361490.4,ENST00000325103.6,ENST000004...",False,ENSG00000107521
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,NaN,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,NaN,NaN,ENSG00000120053.9,GOT1,2121,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,"ENST00000543866.1,ENST00000370508.5",False,ENSG00000120053
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,NaN,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,NaN,NaN,ENSG00000198018.6,ENTPD7,1857,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,ENST00000370489.4,False,ENSG00000198018


## Add whether this is an SE or MXE event

In [21]:
splicing_feature_data['splice_type'] = splicing_feature_data.index.map(lambda x: 'MXE' if len(x.split('@')) == 4 else 'SE')
splicing_feature_data.head()

,Unnamed: 0,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,exon4,exon4_length,gencode_id,gene_name,intron_length,junction12,junction13,junction23,junction24,junction34,strand,transcript_id,exon2_divisible_by_3,one_ensembl_id,splice_type
event_id,,,,,,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,NaN,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,NaN,NaN,ENSG00000107521.14,HPS1,928,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",True,ENSG00000107521,SE
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,NaN,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,NaN,NaN,ENSG00000107521.14,HPS1,3269,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",False,ENSG00000107521,SE
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,NaN,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,NaN,NaN,ENSG00000107521.14,HPS1,1543,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,"ENST00000361490.4,ENST00000325103.6,ENST000004...",False,ENSG00000107521,SE
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,NaN,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,NaN,NaN,ENSG00000120053.9,GOT1,2121,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,"ENST00000543866.1,ENST00000370508.5",False,ENSG00000120053,SE
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,NaN,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,NaN,NaN,ENSG00000198018.6,ENTPD7,1857,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,ENST00000370489.4,False,ENSG00000198018,SE


In [22]:
splicing_feature_data.tail()

,Unnamed: 0,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,exon4,exon4_length,gencode_id,gene_name,intron_length,junction12,junction13,junction23,junction24,junction34,strand,transcript_id,exon2_divisible_by_3,one_ensembl_id,splice_type
event_id,,,,,,,,,,,,,,,,,,,,,,,,,,
exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+,13212,only one,NaN,only one,ENSG00000129824,exon:chrY:2710206-2710283:+,78,exon:chrY:2712118-2712298:+,181,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,172,ENSG00000129824.11,RPS4Y1,12357,chrY:2710284-2712117:+,chrY:2710284-2713686:+,NaN,chrY:2712299-2722640:+,chrY:2713785-2722640:+,+,"ENST00000430575.1,ENST00000250784.8",False,ENSG00000129824,MXE
exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2734834-2734903:+,13213,only one,NaN,only one,ENSG00000129824,exon:chrY:2712118-2712298:+,181,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,172,exon:chrY:2734834-2734903:+,70,ENSG00000129824.11,RPS4Y1,22535,chrY:2712299-2713686:+,chrY:2712299-2722640:+,NaN,chrY:2713785-2734833:+,chrY:2722813-2734833:+,+,"ENST00000430575.1,ENST00000250784.8",False,ENSG00000129824,MXE
exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2734903:+,13214,only one,NaN,only one,ENSG00000129824,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,172,exon:chrY:2733129-2733286:+,158,exon:chrY:2734834-2734903:+,70,ENSG00000129824.11,RPS4Y1,21049,chrY:2713785-2722640:+,chrY:2713785-2733128:+,NaN,chrY:2722813-2734833:+,chrY:2733287-2734833:+,+,"ENST00000430575.1,ENST00000477725.1,ENST000002...",False,ENSG00000129824,MXE
exon:chrY:2803112-2803487:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+,13215,one event with gencode transcripts,NaN,one event with gencode transcripts,ENSG00000067646,exon:chrY:2803112-2803487:+,376,exon:chrY:2821950-2822038:+,89,exon:chrY:2829115-2829687:+,573,exon:chrY:2843136-2843285:+,150,ENSG00000067646.7,ZFY,39648,chrY:2803488-2821949:+,chrY:2803488-2829114:+,NaN,chrY:2822039-2843135:+,chrY:2829688-2843135:+,+,"ENST00000443793.1,ENST00000431102.1,ENST000003...",False,ENSG00000067646,MXE
exon:chrY:2803518-2803810:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+,13216,one event with gencode transcripts,NaN,one event with gencode transcripts,ENSG00000067646,exon:chrY:2803518-2803810:+,293,exon:chrY:2821950-2822038:+,89,exon:chrY:2829115-2829687:+,573,exon:chrY:2843136-2843285:+,150,ENSG00000067646.7,ZFY,39325,chrY:2803811-2821949:+,chrY:2803811-2829114:+,NaN,chrY:2822039-2843135:+,chrY:2829688-2843135:+,+,"ENST00000443793.1,ENST00000431102.1,ENST000003...",False,ENSG00000067646,MXE


## Use only exon ids that were deemed alternative in the data

### Write to a safe place

In [23]:
splicing_feature_data.shape

(36531, 26)

In [24]:
folder

'/projects/ps-yeolab/obotvinnik/singlecell_pnms'

In [25]:
splicing_feature_data.to_csv('{}/splicing_feature_data.csv'.format(csv_folder))